# IMAGE RECOGNITION GENERATOR
## Name of the model : 

First step, you need to enter the name of the model below : 

In [1]:
model_name = "nao"

import os
ROOT_DIR = os.path.abspath(os.curdir)
download_directory = ROOT_DIR + "/PICTURES_DATABASE/"
pics_directory = download_directory  + model_name
print("You work for an image recognition for the", model_name, "inside the directory :", ROOT_DIR)

You work for an image recognition for the nao inside the directory : /home/tonycalvez/GitHub/DEEPDART


## Download : 
First step, you need to download minimum 250 pics with of your model with google images.
You need to find three or four related keywords with the model : 

In [3]:
keywords = ["robot", "school", "humanoide", "aldebaran"]

nb_pics_downloaded_by_keyword = 1 

from google_images_download import google_images_download
response = google_images_download.googleimagesdownload()
for keyword in keywords:
    arguments = {"keywords":model_name + " " + keyword,"limit":nb_pics_downloaded_by_keyword, "print_urls":False, "format":"jpg", "output_directory":download_directory, "image_directory": model_name}
    paths = response.download(arguments)
    


Item no.: 1 --> Item name = nao robot
Evaluating...
Starting Download...
Completed Image ====> 1.robot-humanoide-programmable-nao-v6.jpg

Errors: 0


Item no.: 1 --> Item name = nao school
Evaluating...
Starting Download...
Completed Image ====> 1.NAO.jpg

Errors: 0


Item no.: 1 --> Item name = nao humanoide
Evaluating...
Starting Download...
Completed Image ====> 1.Nao_Robot_%28Robocup_2016%29.jpg

Errors: 0


Item no.: 1 --> Item name = nao aldebaran
Evaluating...
Starting Download...
Completed Image ====> 1.Nao_Robot_%28Robocup_2016%29.jpg

Errors: 0



## Check and Match!
Try to check all pics and verify if all pics have your model.
Moreover it's very important to match your pics and delete the duplicates!

In the bash to install the gthumb software : 
```
sudo apt-get install gthumb 
```

![GTHUMB_DUPLICATES.png](IMG/GTHUMB_DUPLICATES.png)

In [8]:
import os
os.system("gthumb " + pics_directory)

0

This script rename all pictures with a number:

In [49]:
import os
i = 0
for root, dirs, files in os.walk(pics_directory):
    for i,f in enumerate(files):
        absname = os.path.join(root, f)
        newname = os.path.join(root, "data_"+str(i)+".jpg")
        os.rename(absname, newname)
if i < 250:
    print("You have only : ", str(i), "/250 pictures in your database")
    print("Please restart the google image downloader with others keywords")
    
else : 
    print("You have : ", str(i), "pictures! Good Job!")
    

You have only :  7 /250 pictures in your database
Please restart the google image downloader with others keywords


## Scrapping and Labeling your Pics.
In this part, I think this is the worst moment of the project! You need to scrap and label all pictures in your database.
This sofware will generate a YOLO dataset for each picture in the folder. 

In the bash to install the labelImg software : 
```
sudo apt-get install pyqt5-dev-tools
sudo pip install lxml
cd ~/Downloads
git clone https://github.com/tzutalin/labelImg
cd labelImg-master
make qt5py3
```

Please don't forget to change the Pascal VOC data to YOLO data

In [10]:
import os
os.system("labelImg " + pics_directory + " " + model_name)

0

#### YOLO

We need to download the SDK.

In [25]:
import os
ROOT_DIR = os.path.abspath(os.curdir)
DARKNET_DIR = ROOT_DIR + "/darknet/"

pics_directory_darknet = DARKNET_DIR + "VOCdevkit"

os.system("mv " + pics_directory + "/* " + pics_directory_darknet + "/")

0

In [41]:
import glob, os

# Current directo
# Directory where the data will reside, relative to 'darknet.exe'
pics_directory_darknet_relative = 'VOCdevkit/'

# Percentage of images to be used for the test set
percentage_test = 10;

# Create and/or truncate train.txt and test.txt
file_train = open('darknet/cfg/train.txt', 'w')  
file_test = open('darknet/cfg/test.txt', 'w')

# Populate train.txt and test.txt
counter = 1  
index_test = round(100 / percentage_test)  
for pathAndFilename in glob.iglob(os.path.join(pics_directory_darknet, "*.jpg")):  
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:
        counter = 1
        file_test.write(pics_directory_darknet_relative + title + '.jpg' + "\n")
    else:
        file_train.write(pics_directory_darknet_relative + title + '.jpg' + "\n")
        counter = counter + 1
        
file_train.close()
file_test.close() 

In [27]:
os.system("mv " + ROOT_DIR + "/test.txt " + DARKNET_DIR + "cfg/")
os.system("mv " + ROOT_DIR + "/train.txt " + DARKNET_DIR + "cfg/")

0

In [42]:
list_names = ['nao']
nb_class = str(len(list_names))

file_obj = open(DARKNET_DIR + 'cfg/obj.data', 'w')
file_obj.write("classes = " + nb_class + "\ntrain  = cfg/train.txt \nvalid  = cfg/test.txt \nnames = obj.names \nbackup = backup/")
file_obj.close()

file_names = open(DARKNET_DIR + 'cfg/obj.names', 'w')
for name in list_names:
    file_names.write(name + "\n")
file_names.close()
    
os.system("wget https://pjreddie.com/media/files/darknet53.conv.74")

In [46]:
os.system("mv  " + ROOT_DIR + "/darknet53.conv.74 " + DARKNET_DIR + "cfg/")

0

#### Convert Pascal VOC to YOLO
https://ressources.labomedia.org/yolo_sans_carte_graphique
https://timebutt.github.io/static/how-to-train-yolov2-to-detect-custom-objects/
If you don't change your format in YOLO and you have only the Pascal VOC XML, you can find different script Python or you can use an automation of your computer and LabelImg to convert the XML to YOLO dataset.

You need do to an example of the operation :
```
cnee --record --mouse -o /tmp/xnee.xns --first-last --smp --time 1 --stop-key 'a'
```
And when you want do to the repetition:

My script is available in the folder `voctoyolo.sh`


## Generate the information of the database for YOLO
https://pjreddie.com/darknet/yolo/

In this part, we generate the information for the database.
We need some information : 
- training data
- evalutation data










@article{yolov3,
  title={YOLOv3: An Incremental Improvement},
  author={Redmon, Joseph and Farhadi, Ali},
  journal = {arXiv},
  year={2018}
}